# Creating FABnet IPv4 Network

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet.   


## Import the FABlib Library


In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/work/fabric_config/tokens.json
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Bastion Username,pruth_0031379841
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add a separate `l3network` for each site and pass the list of interfaces on that site that you want to connect to FABnetv4. All interfaces passed to `l3network` must be on the same site and each network will be placed on that site.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on different sites you can specify the name of the sites in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a set of random site names that guarantee that the sites are different. 


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [2]:
slice_name = 'MySliceNodeExporterTest2'
[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

Sites: WASH, UCSD


In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Networks
net1 = slice.add_l3network(name=network1_name, type='IPv4')
net2 = slice.add_l3network(name=network2_name, type='IPv4')

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)
node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())
node1.enable_node_exporter(net1)

# Node2
node2 = slice.add_node(name=node2_name, site=site2)
iface2  = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net2.add_interface(iface2)
node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net2.get_gateway())
node2.enable_node_exporter(net2)

#Submit Slice Request
slice.submit();


Retry: 9, Time: 493 sec


ID,47e55e8e-e7b1-43bf-adfb-f0ac040380c8
Name,MySliceNodeExporterTest2
Lease Expiration (UTC),2023-02-25 19:31:56 +0000
Lease Start (UTC),2023-02-24 19:31:57 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
86279ff7-072e-4638-9e48-3c55532181bd,Node1,2,8,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fea9:28d3,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fea9:28d3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d230a537-a702-4205-ab17-e74f45d6e719,Node2,2,8,10,default_rocky_8,qcow2,ucsd-w1.fabric-testbed.net,UCSD,rocky,132.249.252.182,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@132.249.252.182,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
6365adbe-3ee0-4c71-b81e-9b57cd7ca18e,net1,L3,FABNetv4,WASH,10.133.1.0/24,10.133.1.1,Active,
c0943ad8-563a-44b9-ad44-4f2d786079cf,net2,L3,FABNetv4,UCSD,10.134.129.0/24,10.134.129.1,Active,


Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address
Node1-nic1-p1,Node1,net1,100,auto,,02:DF:F2:32:8D:AD,eth1,eth1,10.133.1.2
Node2-nic1-p1,Node2,net2,100,auto,,12:06:C5:B8:0B:7B,eth1,eth1,10.134.129.2



Time to print interfaces 499 seconds


## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [4]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    


PING 10.134.129.2 (10.134.129.2) 56(84) bytes of data.
64 bytes from 10.134.129.2: icmp_seq=1 ttl=59 time=62.5 ms
64 bytes from 10.134.129.2: icmp_seq=2 ttl=59 time=59.7 ms
64 bytes from 10.134.129.2: icmp_seq=3 ttl=59 time=59.7 ms
64 bytes from 10.134.129.2: icmp_seq=4 ttl=59 time=59.7 ms
64 bytes from 10.134.129.2: icmp_seq=5 ttl=59 time=59.7 ms

--- 10.134.129.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 59.691/60.270/62.472/1.133 ms



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()